In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv
import datetime as dt
import re
import time
import dateutil
dataUsageType = '/home/joe/data/datadog/by-usage-type.csv'
dfUsageType = pd.read_csv(dataUsageType)

In [37]:
dfUsageType.head(5)

,UsageType,APS3-DataTransfer-In-Bytes($),USW1-APN2-AWS-Out-Bytes($),USW1-USE2-AWS-In-Bytes($),DeliveryAttempts-SMTP($),USW2-NatGateway-Bytes($),USE1-USE2-AWS-Out-Bytes($),USW2-UGW1-AWS-Out-Bytes($),APS3-RDS:GP2-Storage($),EU-DataTransfer-In-Bytes($),...,JP-Requests-Tier2-HTTPS($),USE2-FreeEventsRecorded($),BoxUsage:t2.xlarge($),us-east-2-KMS-Requests($),USW1-EUW2-AWS-In-Bytes($),RDS:Multi-AZ-StorageUsage($),APS3-APS1-AWS-Out-Bytes($),HeavyUsage:m3.2xlarge($),USE1-EUC1-AWS-Out-Bytes($),Total cost ($)
0,UsageType Total,0,0.017371,0,0,3.800000e-07,171.055233,1.400000e-07,1.965004,0,...,11.641636,0,256.478,0.000042,0,12.053761,0.000014,16024.235543,1694.964273,4.655485e+06
1,Month 3,0,0.001450,0,0,NaN,68.021348,NaN,0.655003,0,...,4.146440,0,3.948,0.000006,0,4.053760,0.000005,5399.649936,613.066400,1.554169e+06
2,Month 2,0,0.007505,0,0,NaN,56.901303,NaN,0.654998,0,...,3.857268,0,98.528,0.000027,0,4.000003,0.000003,5225.467680,556.394938,1.492007e+06
3,Month 1,0,0.008416,0,0,3.800000e-07,46.132582,1.400000e-07,0.655003,0,...,3.637927,0,154.002,0.000009,0,3.999997,0.000006,5399.117927,525.502935,1.609309e+06


In [38]:
dfUsageType.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Columns: 710 entries, UsageType to Total cost ($)
dtypes: float64(559), int64(150), object(1)
memory usage: 22.3+ KB


In [39]:
dataUsageTransposed = '/home/joe/data/datadog/by-usage-transposed.csv'
dfUsgT = pd.read_csv(dataUsageTransposed).fillna(0)
dataEC2InstanceOfferings = '/home/joe/data/datadog/ec2_instance_offerings.txt'
dfEC2Offers = pd.read_csv(dataEC2InstanceOfferings,delimiter="\s+",names=["meta","type","region","meta1"])

In [40]:
dfUsgT

,UsageType,UsageType Total,Month 3,Month 2,Month 1
0,APS3-DataTransfer-In-Bytes($),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,USW1-APN2-AWS-Out-Bytes($),1.737100e-02,1.450070e-03,7.504540e-03,8.416390e-03
2,USW1-USE2-AWS-In-Bytes($),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,DeliveryAttempts-SMTP($),0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,USW2-NatGateway-Bytes($),3.800000e-07,0.000000e+00,0.000000e+00,3.800000e-07
...,...,...,...,...,...
704,RDS:Multi-AZ-StorageUsage($),1.205376e+01,4.053760e+00,4.000003e+00,3.999997e+00
705,APS3-APS1-AWS-Out-Bytes($),1.370000e-05,5.450000e-06,2.700000e-06,5.560000e-06
706,HeavyUsage:m3.2xlarge($),1.602424e+04,5.399650e+03,5.225468e+03,5.399118e+03
707,USE1-EUC1-AWS-Out-Bytes($),1.694964e+03,6.130664e+02,5.563949e+02,5.255029e+02


In [41]:
dfEC2Offers

,meta,type,region,meta1
0,INSTANCETYPEOFFERINGS,x1e.16xlarge,us-east-1,region
1,INSTANCETYPEOFFERINGS,g4dn.2xlarge,us-east-1,region
2,INSTANCETYPEOFFERINGS,r4.4xlarge,us-east-1,region
3,INSTANCETYPEOFFERINGS,r5a.xlarge,us-east-1,region
4,INSTANCETYPEOFFERINGS,t3.small,us-east-1,region
...,...,...,...,...
253,INSTANCETYPEOFFERINGS,r5d.2xlarge,us-east-1,region
254,INSTANCETYPEOFFERINGS,m5a.24xlarge,us-east-1,region
255,INSTANCETYPEOFFERINGS,c5n.18xlarge,us-east-1,region
256,INSTANCETYPEOFFERINGS,r5.metal,us-east-1,region


In [42]:
dfEC2Offers = dfEC2Offers.drop(['meta','region','meta1'], axis=1)
dfEC2Offers

,type
0,x1e.16xlarge
1,g4dn.2xlarge
2,r4.4xlarge
3,r5a.xlarge
4,t3.small
...,...
253,r5d.2xlarge
254,m5a.24xlarge
255,c5n.18xlarge
256,r5.metal
